# Описание проекта  
---
Интернет-магазин ***«Викишоп»*** запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.  
Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.  
Постройте модель со значением метрики качества F1 не меньше 0.75.   

### Описание данных 

- **`text`** — текст комментария 
- **`toxic`** — целевой признак

# Выполнение проекта
---

### Содержание:  
<font size=4><ol>
<li>Получение и первичный анализ данных
</li>
<li>Обучение модели
    <ul>
        <li>2.1. Подготовка признаков</li>
        <li>2.2. Обучение и выбор модели</li>
        <li>2.3. Проверка модели на тестовых данных</li>
    </ul>
</li>
<li>Выводы</li>
</ol></font>

---

### 1. Получение и первичный анализ данных

Импортируем необходимые для работы билиотеки

In [1]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import GridSearchCV
import nltk
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

Получим данные для первичного анализа

In [3]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [4]:
df.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


Удалим лишний столбец

In [5]:
df = df.drop('Unnamed: 0', axis=1)
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


Проверим наличие дубликатов

In [6]:
df.duplicated().sum()

0

Проверим пропуски и типы данных

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Типы данных соответствуют, пропусков нет.  
Проверим баланс классов.

In [8]:
df['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

9:1 - классика.  
Проверим максимальную длину текста.

In [9]:
df['text'].apply(lambda x: len(x)).max()

5000

Многовато, но приемлемо.

## 2. Обучение модели

### 2.1. Подготовка признаков

Избавим тексты от лишних символов, оставим только английские буквы и пробелы, сохраним их в новый столбец.

In [10]:
df['re_texts'] = df['text'].apply(lambda x: ' '.join(re.sub(r"[^a-zA-Z' ]", ' ', x).split()))

Приведем слова к лемме. Для этого загрузим необходимые пакеты и создадим лемматизатор.

In [11]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/zalinaramazanova/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/zalinaramazanova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/zalinaramazanova/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Опишем процесс функциями, т.к. нам в процессе будет несколько этапов - нам будет нужен pos-tag для каждого слова, означающий часть речи.

In [12]:
def correct_tag(tag):
    tag_dict = {'NN':'n', 'JJ':'a', 'VB':'v', 'RB':'r'}
    try:
        return tag_dict[tag[:2]]
    except:
        return 'n' 


def lemmatize_sent(row): 
    return ' '.join([lemmatizer.lemmatize(word.lower(), pos=correct_tag(tag)) 
                     for word, tag in pos_tag(word_tokenize(row))])

In [13]:
df['lemms'] = df['re_texts'].apply(lemmatize_sent)

In [14]:
df.head(10)

,text,toxic,re_texts,lemms
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,D'aww He matches this background colour I'm se...,d'aww he match this background colour i 'm see...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I'm really not trying to edit war It's...,hey man i 'm really not try to edit war it 's ...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can't make any real suggestions on impr...,more i ca n't make any real suggestion on impr...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...,you sir be my hero any chance you remember wha...
5,"""\n\nCongratulations from me as well, use the ...",0,Congratulations from me as well use the tools ...,congratulation from me a well use the tool wel...
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,cocksucker before you piss around on my work
7,Your vandalism to the Matt Shirvington article...,0,Your vandalism to the Matt Shirvington article...,your vandalism to the matt shirvington article...
8,Sorry if the word 'nonsense' was offensive to ...,0,Sorry if the word 'nonsense' was offensive to ...,sorry if the word 'nonsense ' be offensive to ...
9,alignment on this subject and which are contra...,0,alignment on this subject and which are contra...,alignment on this subject and which be contrar...


Теперь составим TF-IDF каждого слова в строке. Ограничим размер 3000, иначе память не выдерживает.

In [15]:
nltk.download('stopwords')
stopwords = list(set(nltk.corpus.stopwords.words('english')))

count_tf_idf = TfidfVectorizer(stop_words=stopwords) #, max_features=3000)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zalinaramazanova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
features = df['lemms']
target = df['toxic']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.1, random_state=123)

print(features_train.shape)
print(target_train.value_counts(normalize=True))

(143362,)
0    0.898048
1    0.101952
Name: toxic, dtype: float64


In [17]:
tf_idf_train = count_tf_idf.fit_transform(features_train)

In [18]:
tf_idf_train.shape

(143362, 147346)

Добавим новые значения в таблицу в виде отдельных столбцов. 

### 2.2. Обучение и выбор модели

Перед нами классическая задача бинарной классификации. Будем использовать модели логистической регрессии и градиентного бустинга.  

Создадим объект, попробуем установить гиперпараметр max_iter повыше чтобы алгоритм обучался корректно. Так же нам важен баланс классов.

In [19]:
log_reg = LogisticRegression(class_weight='balanced', max_iter=1000)

Проверим точность работы модели с помощью кросс-валидации.

In [20]:
grid_log_reg = GridSearchCV(estimator=log_reg, n_jobs=-1, cv=5, scoring='f1',
                            param_grid={'C': [i for i in np.arange(0.5, 12.5, 0.1)]})

In [21]:
grid_log_reg.fit(tf_idf_train, target_train)

print('Лучший результат модели = {:.3f}'.format(grid_log_reg.best_score_)) 
print('Лучшие параметры модели =', grid_log_reg.best_params_)

Лучший результат модели = 0.761
Лучшие параметры модели = {'C': 6.699999999999998}


В порог уже уложились, отлично. Рассмотрим CatBoost

In [30]:
catboost_model = CatBoostClassifier(random_state=123, learning_rate=0.5, max_depth=10, n_estimators=100)

In [31]:
cv_cb = cross_val_score(estimator=catboost_model, 
                        X=tf_idf_train, 
                        y=target_train, 
                        scoring='f1', 
                        n_jobs=-1)

0:	learn: 0.3470704	total: 17.9s	remaining: 29m 30s
0:	learn: 0.3297168	total: 18.1s	remaining: 29m 48s
0:	learn: 0.3365819	total: 18.4s	remaining: 30m 20s
0:	learn: 0.3464886	total: 18.3s	remaining: 30m 11s
0:	learn: 0.3386854	total: 18.3s	remaining: 30m 10s
1:	learn: 0.2514513	total: 36.3s	remaining: 29m 38s
1:	learn: 0.2457385	total: 36.6s	remaining: 29m 55s
1:	learn: 0.2508319	total: 37s	remaining: 30m 14s
1:	learn: 0.2461256	total: 37.1s	remaining: 30m 15s
1:	learn: 0.2523971	total: 37.1s	remaining: 30m 16s
2:	learn: 0.2232475	total: 55.2s	remaining: 29m 45s
2:	learn: 0.2182424	total: 55.9s	remaining: 30m 7s
2:	learn: 0.2202134	total: 55.9s	remaining: 30m 7s
2:	learn: 0.2245211	total: 56.5s	remaining: 30m 25s
2:	learn: 0.2187492	total: 56.6s	remaining: 30m 29s
3:	learn: 0.2071679	total: 1m 14s	remaining: 29m 58s
3:	learn: 0.2061702	total: 1m 15s	remaining: 30m 21s
3:	learn: 0.2077057	total: 1m 15s	remaining: 30m 20s
3:	learn: 0.2031533	total: 1m 16s	remaining: 30m 39s
3:	learn: 0.

In [32]:
print('Среднее значение F1 CatBoost на кросс-валидации = {:.3f}'.format(cv_cb.mean()))

Среднее значение F1 CatBoost на кросс-валидации = 0.750


Необходимое значение достигнуто, но уступает логистической регрессии.

CatBoost так же позволяет обрабатывать тексты без векторизации - посмотрим что из этого получится. В качестве признака используем столбец с леммами, для оценки модели будем так же использовать кросс-валидацию.

Установим гиперпараметры - количество оценщиков, максимальную глубину дерева, и шаг бустинга.

In [33]:
catboost_text_model = CatBoostClassifier(text_features=['lemms'],
                                         learning_rate=0.6, 
                                         random_state=123, 
                                         max_depth=10, 
                                         n_estimators=100)

Посчитаем значения метрики F1 на кросс-валидации.

In [34]:
cvs = cross_val_score(estimator=catboost_text_model, 
                      X=pd.DataFrame(features_train), 
                      y=target_train, 
                      scoring='f1', 
                      n_jobs=-1)

0:	learn: 0.3450968	total: 4.55s	remaining: 7m 30s
0:	learn: 0.3445495	total: 5.33s	remaining: 8m 47s
0:	learn: 0.3462879	total: 5.49s	remaining: 9m 3s
0:	learn: 0.3440072	total: 5.42s	remaining: 8m 56s
1:	learn: 0.2362887	total: 9.74s	remaining: 7m 57s
1:	learn: 0.2365793	total: 10.8s	remaining: 8m 47s
1:	learn: 0.2365146	total: 11.2s	remaining: 9m 10s
1:	learn: 0.2346418	total: 11.1s	remaining: 9m 6s
2:	learn: 0.1867418	total: 15.6s	remaining: 8m 24s
0:	learn: 0.3435470	total: 6.18s	remaining: 10m 11s
2:	learn: 0.1868330	total: 17.1s	remaining: 9m 13s
2:	learn: 0.1869601	total: 17.7s	remaining: 9m 31s
2:	learn: 0.1860144	total: 17.7s	remaining: 9m 33s
3:	learn: 0.1632425	total: 21.9s	remaining: 8m 45s
1:	learn: 0.2344391	total: 12.6s	remaining: 10m 16s
3:	learn: 0.1622611	total: 23.3s	remaining: 9m 18s
3:	learn: 0.1626156	total: 23.9s	remaining: 9m 32s
3:	learn: 0.1615286	total: 23.8s	remaining: 9m 31s
4:	learn: 0.1522370	total: 28.2s	remaining: 8m 56s
2:	learn: 0.1873539	total: 18.8

In [35]:
print('Значение F1 на кросс-валидации по сырым текстам = {:.3f}'.format(cvs.mean()))

Значение F1 на кросс-валидации по сырым текстам = 0.780


Это победа.

Поскольку самые лучшие результаты показала модель CatBoostClassifier на текстовых данных, то обучим её.

In [36]:
catboost_text_model.fit(X=pd.DataFrame(features_train), y=target_train)

0:	learn: 0.3463370	total: 1.04s	remaining: 1m 42s
1:	learn: 0.2352991	total: 2s	remaining: 1m 38s
2:	learn: 0.1862056	total: 3s	remaining: 1m 36s
3:	learn: 0.1608484	total: 4.09s	remaining: 1m 38s
4:	learn: 0.1482539	total: 5.12s	remaining: 1m 37s
5:	learn: 0.1413944	total: 6.1s	remaining: 1m 35s
6:	learn: 0.1371936	total: 7.17s	remaining: 1m 35s
7:	learn: 0.1345782	total: 8.28s	remaining: 1m 35s
8:	learn: 0.1308831	total: 9.21s	remaining: 1m 33s
9:	learn: 0.1293474	total: 10.2s	remaining: 1m 32s
10:	learn: 0.1276521	total: 11.3s	remaining: 1m 31s
11:	learn: 0.1265204	total: 12.3s	remaining: 1m 30s
12:	learn: 0.1254745	total: 13.4s	remaining: 1m 29s
13:	learn: 0.1239733	total: 14.5s	remaining: 1m 28s
14:	learn: 0.1231555	total: 15.5s	remaining: 1m 27s
15:	learn: 0.1226205	total: 16.5s	remaining: 1m 26s
16:	learn: 0.1209520	total: 17.6s	remaining: 1m 25s
17:	learn: 0.1204438	total: 18.6s	remaining: 1m 24s
18:	learn: 0.1198016	total: 19.6s	remaining: 1m 23s
19:	learn: 0.1190061	total: 2

### 2.3. Проверка модели на тестовых данных

Проверим значение метрики F1 на тестовых данных.

In [37]:
print('Метрика F1 на тестовых данных = {:.3f}'.format(
    f1_score(y_true=target_test, y_pred=catboost_text_model.predict(pd.DataFrame(features_test)))))

Метрика F1 на тестовых данных = 0.787


Пороговое значение в 0.75 преодолено, модель работает корректно.

## 3. Выводы

Модель градиентного бустинга **`CatBoostClassifier`** хорошо справилась с задачей без предварительной токенизации и векторизации текста. Текст был лишь очищен с помощью регулярных выражений и приведен к лемме. Приводить классы к балансу не понадобилось, метрики качества были достаточны. Результаты лучше чем у признаков TF-IDF.  

Результаты метрики F1 на кросс-валидации:  
- Логистическая регрессия - 0.761
- CatBoost (TF-IDF) - 0.750
- CatBoost (текст) - 0.780

Результаты модели CatBoostClassifier (сырые тексты):  
- F1 на тестовых данных = 0.787.  

Гиперпараметры:  
- максимальная глубина - 10, 
- количество оценщиков - 100, 
- шаг бустинга - 0.6.